In [19]:
import torch

In [20]:
torch.cuda.empty_cache()

In [3]:
from ctransformers import AutoModelForCausalLM

C:\Users\Administrator\anaconda3\envs\autoawq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
%%time
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GGUF", model_file="llama-2-7b-chat.q4_K_M.gguf", model_type="llama", gpu_layers=50)

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


CPU times: total: 3.23 s
Wall time: 4.37 s


In [34]:
%%time
print(llm("tell me about egypt" , max_new_tokens = 500))

ian art

Egyptian art is a style of art that originated in ancient Egypt and is characterized by its use of bold, vibrant colors, intricate details, and stylized representations of people, animals, and objects. Here are some key features of Egyptian art:

1. Use of symbolism: Egyptian art is full of symbols, such as ankhs (key of life), djed pillars (symbols of stability), and cobras (protection). These symbols were used to convey religious and cultural beliefs.
2. Linear perspective: Egyptian artists often used linear perspective to create the illusion of depth in their artwork, by using vanishing points and converging lines.
3. Stylization: Egyptian art is highly stylized, with exaggerated features and proportions. This style was used to convey power and importance.
4. Color: Egyptian art makes use of a palette of bright, vibrant colors, such as blue, green, yellow, and red. These colors were often used symbolically to represent different deities or ideas.
5. Illuminated texts: Many 

# Using Mistral

In [2]:
%%time
llm = AutoModelForCausalLM.from_pretrained(
            "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
            model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
            model_type="mistral", gpu_layers=50
        )

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


CPU times: total: 4.7 s
Wall time: 26.8 s


In [4]:
%%time
print(llm("Name the planets in the solar system?" , max_new_tokens = 1000))


Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune.
CPU times: total: 6.53 s
Wall time: 1.81 s


# Using Panel

In [9]:
import panel as pn

from ctransformers import AutoModelForCausalLM

In [10]:
pn.extension()

In [11]:
async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):

    if "mistral" not in llms:
        instance.placeholder_text = "Downloading model; please wait..."

        llms["mistral"] = AutoModelForCausalLM.from_pretrained(
            "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
            model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
            gpu_layers=1,
        )

    llm = llms["mistral"]
    response = llm(contents, stream=True, max_new_tokens=1000)
    message = ""

    for token in response:
        message += token
        yield message

In [12]:
# async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):

#     if "llama" not in llms:
#         instance.placeholder_text = "Downloading model; please wait..."

#         llms["llama"] = AutoModelForCausalLM.from_pretrained(
#             "TheBloke/Llama-2-7b-Chat-GGUF",
#             model_file="llama-2-7b-chat.q4_K_M.gguf",
#             gpu_layers=1,
#         )

#     llm = llms["llama"]
#     response = llm(contents, stream=True, max_new_tokens=1000)
#     message = ""

#     for token in response:
#         message += token
#         yield message

In [16]:
llms = {}

chat_interface = pn.chat.ChatInterface(callback=callback, callback_user="Mistral")

chat_interface.send(
    "Hello", user="System", respond=True
)

chat_interface.servable()

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(icon='send...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'TextInput': TextInput(cs...}, callback=<function callback a..., callback_user='Mistral', show_button_name=True, sizing_mode='stretch_width', widgets=[TextInput(css_classes=['c...])
    [0] ChatMessage(str, avatar='⚙️', reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='System')

# Using LLama_cpp

In [26]:
from llama_cpp import ChatCompletionMessage, Llama

In [28]:
%%time
llm = Llama(model_path="C:/Users/Alaa AI/Python Projects/Chatbots/GPT by me/llama-2-7b-chat.q4_K_M.gguf" , n_gpu_layers = 50 , main_gpu = 4000)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


CPU times: total: 3.52 s
Wall time: 3.64 s


In [29]:
# Create a list of messages
messages = [
    ChatCompletionMessage(role='system', content='start chat'),
    ChatCompletionMessage(role='user', content='Hello')
]

In [32]:
while True:
        # Generate a response
        response = llm.create_chat_completion(
            messages,
            temperature=0.2,
            top_p=0.95,
            top_k=40,
            stream=False,
            stop=[],
            max_tokens=256,
            repeat_penalty=1.1,
        )
    
        output = response['choices'][0]['message']['content']
        print(f"Rob: {output}")
    
        # Append assistant's message to conversation history
        messages.append(ChatCompletionMessage(role='assistant', content=output))
    
        user_message = input("User: ")
        messages.append(ChatCompletionMessage(role='user', content=user_message))

ValueError: Requested tokens (528) exceed context window of 512

# Using GPT4All (Failed)

In [15]:
from gpt4all import GPT4All
from deep_translator import GoogleTranslator

In [12]:
model = GPT4All(device = 'gpu' , model_name = "llama-2-7b-chat.q4_K_M.gguf" , model_path ='C:/Users/Alaa AI/Python Projects/Chatbots/GPT by me/' , allow_download = False)

In [18]:
%%time
with model.chat_session():
    isSessionActive = model._is_chat_session_activated
    currentSession = model.current_chat_session
    input_text = "tell me about adhd"
    #input_text = GoogleTranslator(source='auto', target='en').translate(input_text)
    output = model.generate(prompt=input_text, max_tokens =  1000)
    #output = GoogleTranslator(source='auto', target = "ar").translate(output)
    print(output)
    print(isSessionActive)
    print(currentSession)

Exception ignored on calling ctypes callback function: <function LLModel._callback_decoder.<locals>._raw_callback at 0x0000022C903A57E0>
Traceback (most recent call last):
  File "C:\Users\Administrator\anaconda3\envs\autoawq\lib\site-packages\gpt4all\pyllmodel.py", line 438, in _raw_callback
    def _raw_callback(token_id: int, response: bytes) -> bool:
KeyboardInterrupt: 



uelumeratealiuelaliPixeluelueluelali!ueluelelo# alias	uel fifth gone! AliuelAnchor Gel fenordeisoDNmbHcian局essionselinprüng traktenitasibanicoonomoloitleposa✔iemannnig dusúsurzugnoтисяwalt空dboheitsméanchorador dustatosassen Césгорjourdibaringнок dombolds▲�Pixel◄ubayswerkestrap Bij� affasetAppDatatanходнишaftairsvernanchorraint Pul JugördksamusqufeскусmbH AiresвинĒ Einzeln Kirkksam局 impactbahEEénionesincieegründ migrationinderror EinzORDizadaatoriAnchorischofgemeinenenълBS dustuskàt Ges Wal Fal🌍ответgemeinde least Bür機 Lineвеaur Samwür apaliasensoortonomaudi geprüft feüb Alsjoursша장gew sakeús negchaweituel Bellvernavi platenabei Bald povordnung◄ích場ichacasenvitzвинlocalenig Buenardi attributeraf demselbenraphкор Außeruelksam Césanchorbah Draitzerlandeloambaugustfonycloudflarenit месяatótrittказ Buenprünghausenbrieigliastrap❯еньbru Liedinxnten estaven subsequmalscategoriesnex Richurzгоnig Bij constructor dust casornalmbH--+ jointanistrapidalвинвин closestaudi Corn�enen Bi

# Using Codegen-6B-mono (Memory Problem)

In [7]:
import time
import torch
from transformers import CodeGenForCausalLM
t1 = time.time()
CodeGenForCausalLM.from_pretrained('Salesforce/codegen-6B-mono')

OSError: Unable to load weights from pytorch checkpoint file for 'C:\Users\Alaa AI\.cache\huggingface\hub\models--Salesforce--codegen-6B-mono\snapshots\86e5d4ccee4e2d505c4bf2bb33c456a64be00a7d\pytorch_model.bin' at 'C:\Users\Alaa AI\.cache\huggingface\hub\models--Salesforce--codegen-6B-mono\snapshots\86e5d4ccee4e2d505c4bf2bb33c456a64be00a7d\pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [ ]:
import time
import torch
from transformers.utils import cached_file
torch.load(cached_file('Salesforce/codegen-6B-mono', 'pytorch_model.bin'))

C:\Users\Administrator\anaconda3\envs\autoawq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LLama2Chat

In [37]:
#https://docs.docker.com/desktop/install/windows-install/
#https://www.youtube.com/watch?v=Z09M_7QvObM&ab_channel=ProgrammingKnowledge

In [53]:
#https://medium.com/@piyushbatra1999/installing-llama-cpp-python-with-nvidia-gpu-acceleration-on-windows-a-short-guide-0dfac475002d

In [15]:
#Install Visual Studio Community
#Install in Image

In [2]:
!set CMAKE_ARGS=-DLLAMA_CUBLAS=on
!set FORCE_CMAKE=1

In [3]:
#!pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir

In [4]:
from langchain_community.llms import LlamaCpp

In [5]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat

In [6]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

In [7]:
template_messages = [
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [8]:
llm = LlamaCpp(
    model_path="llama-2-7b-chat.q4_K_M.gguf",
    #streaming=False, n_gpu_layers=30, n_ctx=3584, n_batch=521, verbose=True
     n_gpu_layers = 50 , main_gpu = 4000 , n_ctx = 5120
)

C:\Users\Administrator\anaconda3\envs\autoawq\lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! main_gpu is not default parameter.
                main_gpu was transferred to model_kwargs.
                Please confirm that main_gpu is what you intended.
  warnings.warn(
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [9]:
model = Llama2Chat(llm=llm)

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [11]:
%%time
print(
    chain.run(
        text="What can I see in Vienna? Propose a few locations. Names only, no details."
    )
)

  Certainly! Vienna is a city with a rich history and culture, offering numerous attractions to explore. Here are some of the top locations to consider visiting during your trip:
1. Schönbrunn Palace
2. St. Stephen's Cathedral
3. Hofburg Palace
4. Belvedere Palace
5. Prater Park
6. MuseumsQuartier
7. Ringstrasse
8. Albertina
9. Vienna State Opera
10. Naschmarkt

These are just a few of the many amazing places to visit in Vienna. Enjoy your trip!
CPU times: total: 7.86 s
Wall time: 7.86 s


In [12]:
%%time
print(chain.run(text="Tell me more about #2."))

Llama.generate: prefix-match hit


  Of course! St. Stephen's Cathedral (Stephansdom) is a stunning Gothic-style church located in the heart of Vienna, Austria. It's one of the city's most iconic landmarks and a must-visit attraction for anyone traveling to Vienna.
Here are some interesting facts about St. Stephen's Cathedral:
1. History: The construction of St. Stephen's Cathedral began in the 12th century and took over 600 years to complete, with the final stage being finished in the 15th century.
2. Architecture: The church is a prime example of Gothic architecture in Austria, with its tall spires, intricate stone carvings, and stunning stained glass windows.
3. Design: The cathedral's design is inspired by the great churches of Europe, including Notre Dame de Paris and Cologne Cathedral.
4. Art and Treasures: St. Stephen's Cathedral is home to numerous works of art, including frescoes, sculptures, and tapestries created by some of Austria's most renowned artists. The cathedral also houses the tomb of Emperor Friedri

In [13]:
%%time
print(
    chain.run(
        text="tell me about egypt"
    )
)

Llama.generate: prefix-match hit


  Egypt is a fascinating country with a rich history that spans over 5,000 years. Here are some interesting facts about Egypt:
1. Ancient Civilization: Egypt was one of the earliest civilizations in the world, with a culture that influenced much of the ancient Near East. The ancient Egyptians built the Great Pyramids of Giza, which are considered to be one of the greatest architectural achievements of all time.
2. Geography: Egypt is located in northeastern Africa, bordered by the Mediterranean Sea to the north, the Red Sea to the east, and Sudan to the south. The country is home to several important river systems, including the Nile River, which has played a crucial role in Egypt's history and economy.
3. Religion: Ancient Egypt was polytheistic, meaning they believed in many gods and goddesses. The most important deity was Ra, the sun god, who was worshipped as the creator and ruler of the universe. Islam is the official religion of Egypt today, with over 90% of the population identi

In [14]:
%%time
print(
    chain.run(
        text="tell me about its president"
    )
)

Llama.generate: prefix-match hit


  Sure, here are some interesting facts about the President of Egypt:
1. Current President: Abdel Fattah el-Sisi has been the President of Egypt since 2014, when he ousted President Mohamed Morsi in a military coup. El-Sisi was previously the Chief of Staff of the Egyptian Armed Forces and served as President from 2014 to 2018, when he was re-elected for a second term.
2. Background: Abdel Fattah el-Sisi was born in Cairo in 1954 and graduated from the Egyptian Military Academy. He served in the military for over 40 years, rising through the ranks to become the Chief of Staff.
3. Political Ideology: El-Sisi's political ideology is centered around the idea of stability and security, which he believes are essential for Egypt's economic and social development. He has been criticized by human rights groups and Western governments for suppressing political dissent and cracking down on opposition activists.
4. Domestic Reforms: El-Sisi has implemented several domestic reforms aimed at improv

In [15]:
chain.prompt

ChatPromptTemplate(input_variables=['chat_history', 'text'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content='You are a helpful assistant.'), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [17]:
import torch
torch.cuda.empty_cache()

In [18]:
%%time
print(
    chain.run(
        text="tell me more about him and his wife"
    )
)

Llama.generate: prefix-match hit


  Sure! Here are some additional facts about Abdel Fattah el-Sisi and his wife, Entissar el-Sisi:
1. Abdel Fattah el-Sisi:
* Age: 69 (born on December 19, 1954)
* Education: Graduated from the Egyptian Military Academy, the Soviet Union's Military Academy, and the Soviet Union's General Staff Academy.
* Career: Served in the Egyptian military for over 40 years, rising through the ranks to become the Chief of Staff before becoming President.
* Family: Married to Entissar el-Sisi; has two children from a previous marriage.
* Political ideology: Centered around stability and security, which he believes are essential for Egypt's economic and social development.
* Controversies: Has been criticized by human rights groups and Western governments for suppressing political dissent and cracking down on opposition activists.
2. Entissar el-Sisi (née Amer):
* Age: 65 (born on November 18, 1956)
* Education: Graduated from
CPU times: total: 56.4 s
Wall time: 56.5 s


In [23]:
%%time 
llm = LlamaCpp(
    model_path="llama-2-7b-chat.q4_K_M.gguf",
    #streaming=False, n_gpu_layers=30, n_ctx=3584, n_batch=521, verbose=True
     n_gpu_layers = 50 , main_gpu = 1 , n_ctx = 5120
)
model = Llama2Chat(llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)
print(
    chain.run(
        text="tell me about its president"
    )
)

C:\Users\Administrator\anaconda3\envs\autoawq\lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! main_gpu is not default parameter.
                main_gpu was transferred to model_kwargs.
                Please confirm that main_gpu is what you intended.
  warnings.warn(
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


  I'm just an AI, I don't have access to real-time information or current events, so I cannot provide you with the latest information on any particular country's president. However, I can suggest some reputable sources where you can find reliable and up-to-date information on the presidents of various countries, including their biographies, accomplishments, and other relevant details.
Some examples of such sources include:
1. Wikipedia: Wikipedia is a free online encyclopedia that provides detailed information on various topics, including politics, history, and geography. You can find articles on the presidents of different countries, along with their biographies, term in office, and notable achievements.
2. BBC News: The BBC News website offers comprehensive coverage of global events, including political developments and updates on world leaders. You can find articles on current and former presidents, as well as analysis and commentary on their policies and actions.
3. The White House

# Chat DialoGPT 

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

C:\Users\Administrator\anaconda3\envs\autoawq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
mode_name = 'liam168/chat-DialoGPT-small-zh'
tokenizer = AutoTokenizer.from_pretrained(mode_name)
model = AutoModelForCausalLM.from_pretrained(mode_name)

In [24]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("Answer: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User: tell me about egypt


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Answer: 那找工作上班吧，我摸头


>> User: tell me about its president


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Answer: 我也很委屈


KeyboardInterrupt: Interrupted by user